In [14]:
# Import necessary libraries and packages
import math
import numpy as np
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

from qiskit import Aer, IBMQ, QuantumCircuit
from qiskit.primitives import Estimator
from qiskit.utils import QuantumInstance
from qiskit.circuit import QuantumCircuit, Parameter, ParameterVector
from qiskit.opflow import PauliSumOp, commutator, CircuitStateFn
from qiskit.opflow.converters import AbelianGrouper
from qiskit.compiler import transpile

# Import Qiskit libraries for VQE
from qiskit.algorithms import NumPyMinimumEigensolver, VQE
from qiskit.algorithms.optimizers import SLSQP, SPSA, COBYLA

# Import Qiskit Nature libraries
# from qiskit_nature.algorithms import GroundStateEigensolver, VQEUCCFactory
# from qiskit_nature.circuit.library import UCC, UCCSD
# from qiskit_nature.drivers import Molecule
# from qiskit_nature.drivers.second_quantization import (ElectronicStructureDriverType,
#                                                        ElectronicStructureMoleculeDriver)
# from qiskit_nature.converters.second_quantization import QubitConverter
# from qiskit_nature.mappers.second_quantization import BravyiKitaevMapper, JordanWignerMapper, ParityMapper
# from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
# from qiskit_nature.transformers.second_quantization.electronic import (ActiveSpaceTransformer,
#                                                                        FreezeCoreTransformer)

# from qiskit_nature.settings import settings
# settings.dict_aux_operators = True

In [17]:
def get_qubit_op(dist):
    
    geom = [["H", [0.0, 0.0, 0.0]], ["Li", [0.0, 0.0, dist]]]
    lih = Molecule(geometry=geom, multiplicity=1, charge=0)
    driver = ElectronicStructureMoleculeDriver(lih,
                                               basis="sto3g",
                                               driver_type=ElectronicStructureDriverType.PYSCF) 
    
    problem = ElectronicStructureProblem(driver, [FreezeCoreTransformer(True, [3, 4])])
    converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True)
    
    second_q_ops = problem.second_q_ops()
    # map to qubit operators
    qubit_op = converter.convert(second_q_ops.get('ElectronicEnergy'), num_particles=problem.num_particles)
    
    return qubit_op, converter, problem

In [18]:
from itertools import product    

def get_binary(length, x, y):
    perm=product([x, y], repeat=length)
    possible_bin=[]
    for i in list(perm):  
        my_bin=''.join(i) 
        possible_bin.append(my_bin)
    return possible_bin


def get_flip_index(pauli: str):
    
    string = ''
    num_qubits = len(pauli)
    for i in pauli:
        if i == 'X' or i == 'Y':
            string += '1'
        else:
            string += '0'
    
    if len(string) != num_qubits:
        raise 'Error of length'
    
    return string

In [19]:
def Hamiltonian_grouper(qubit_op):
    '''This function will group the qubit Hamiltonian into separate groups according to their flip indices.'''
    
    num_qubits = qubit_op.num_qubits
    keys_list = get_binary(num_qubits, '0', '1')
    
    Hamiltonian_list = []
    for i in qubit_op:
        Hamiltonian_list.append(i.primitive.to_list()[0])
    
    # We create a dictionary to store each group as a PauliSumOperator, where the key is a string of '0's and
    # '1's, where the '1's denote the flip indices.
    H_Dict = {}
    
    for i in keys_list:
        pauli_list = []
        for j in Hamiltonian_list:
            pauli = j[0]
            if get_flip_index(pauli) == i:
                pauli_list.append(j)
                
                # This algorithm can be accelated by removing the element being selected.
        if pauli_list != []:
            H_Dict.update({i: PauliSumOp.from_list(pauli_list)})
    
    return H_Dict

In [20]:
def compute_gradient(H_Dict, circuit):
    
    '''This function computes the gradient of energy functional wrt a representative Pauli word from each group.
    This implementation now only works with statevector simulator.'''
    
    # First we create a dictionary to store the gradient
    gradient = {}
    for i in H_Dict.keys():
        num_flip_indices = i.count('1')
        
        if num_flip_indices > 0:
            pauli = ''
            for j in i:
                if j == '0':
                    pauli += 'I'
                else:
                    pauli += 'X'
                    
            for j in reversed(range(len(pauli))):
                pauli = list(pauli)
                if pauli[j] == 'X':
                    pauli[j] = 'Y'
                    break
                    
            pauli = ''.join(pauli)
            pauli_list = [(pauli, 1.0 + 1.0j)]
            P = PauliSumOp.from_list(pauli_list)
            operator = H_Dict.get(i)@P            # according to Eqn 9. from JCTC. 2020, 16, 1055-1063
            
            psi = CircuitStateFn(circuit)
            score = psi.adjoint().compose(operator).compose(psi).eval().imag
            if abs(score) > 1E-7:
                gradient.update({pauli: abs(score)})
        
    return gradient

In [30]:
def construct_qcc_circuit(entanglers: list, backend, truncation=None):
    '''This function defines the QCC ansatz circuit for VQE. Here we construct exponential blocks using
    entanglers from QMF state as a proof of principle demonstration.
    
    Args:
        entanglers: list storing Pauli words for construction of qcc_circuit.
        backend: statevector, qasm simulator or a real backend.
        truncation: a threshold number to truncate the blocks. Default: None.
    Returns:
        qcc_circuit
    '''
    
    if truncation != None:
        if len(entanglers) > truncation:
            num_blocks = truncation
        else:
            num_blocks = len(entanglers)
    else:
        num_blocks = len(entanglers)
    
    p = ParameterVector('p', num_blocks)
    
    num_qubits = len(entanglers[0])
    qcc_circuit = QuantumCircuit(num_qubits)
    for i in range(num_blocks):
        circuit = QuantumCircuit(num_qubits)
        key = entanglers[i]
        coupler_map = []
        
        # We first construct coupler_map according to the key.
        for j in range(num_qubits):
            if key[num_qubits-1-j] != 'I':
                coupler_map.append(j)
                
        # Then we construct the circuit.
        if len(coupler_map) == 1:
            # there is no CNOT gate.
            c = coupler_map[0]
            if key[num_qubits-1-c] == 'X':
                circuit.h(c)
                circuit.rz(p[i], c)
                circuit.h(c)
            elif key[num_qubits-1-c] == 'Y':
                circuit.rx(-np.pi/2, c)
                circuit.rz(p[i], c)
                circuit.rx(np.pi/2, c)

            qcc_circuit += circuit

        else:
            # Here we would need CNOT gate.
            for j in coupler_map:
                # layer 1
                if key[num_qubits-1-j] == 'X':
                    circuit.h(j)
                elif key[num_qubits-1-j] == 'Y':
                    circuit.rx(-np.pi/2, j)
                    
            for j in range(len(coupler_map) - 1):
                circuit.cx(coupler_map[j], coupler_map[j+1])
                
            param_gate = QuantumCircuit(num_qubits)
            param_gate.rz(p[i], coupler_map[-1])
            # qcc_circuit += circuit + param_gate + circuit.inverse()
            qcc_circuit.compose(circuit, inplace=True)
            qcc_circuit.compose(param_gate, inplace=True)
            qcc_circuit.compose(circuit.inverse(), inplace=True)
    
    
    trans_circuit = transpile(qcc_circuit, backend=backend, optimization_level=3)
    
    return trans_circuit

In [31]:
backend = Aer.get_backend("statevector_simulator")
entanglers = ['IXXY', 'ZYZX', 'IXYX', 'ZXXY', 'XYZX', 'ZXYX', 'YXZX']

qcc_circuit = construct_qcc_circuit(entanglers, backend)

qcc_circuit.draw()


global phase: 3π/2
     ┌──────────┐                                        ┌────────────┐     »
q_0: ┤ Rx(-π/2) ├──■──────────────────────────────■──────┤ U2(π/2,-π) ├──■──»
     └──┬───┬───┘┌─┴─┐                          ┌─┴─┐    └───┬───┬────┘┌─┴─┐»
q_1: ───┤ H ├────┤ X ├──■────────────────■──────┤ X ├────────┤ H ├─────┤ X ├»
        ├───┤    └───┘┌─┴─┐┌──────────┐┌─┴─┐┌───┴───┴───┐    └───┘     └───┘»
q_2: ───┤ H ├─────────┤ X ├┤ Rz(p[0]) ├┤ X ├┤ U2(0,π/2) ├───────────────────»
        └───┘         └───┘└──────────┘└───┘└───────────┘                   »
q_3: ───────────────────────────────────────────────────────────────────────»
                                                                            »
«                                                                              »
«q_0: ──────────────────────────────────────────────────────────────────────■──»
«                                      ┌──────────┐                       ┌─┴─┐»
«q_1: ──■──────────────────────────■───┤ Rx(-π/2) ├───■────────────────■──┤ X ├»
«     ┌─┴─┐                      ┌─┴─┐┌┴──────────┴┐┌─┴─┐┌──────────┐┌─┴─┐└───┘»
«q_2: ┤ X ├──■────────────────■──┤ X ├┤ U2(π/2,-π) ├┤ X ├┤ Rz(p[2]) ├┤ X ├─────»
«     └───┘┌─┴─┐┌──────────┐┌─┴─┐└───┘└────────────┘└───┘└──────────┘└───┘     »
«q_3: ─────┤ X ├┤ Rz(p[1]) ├┤ X ├──────────────────────────────────────────────»
«          └───┘└──────────┘└───┘                                              »
«     ┌───────────┐                                                   »
«q_0: ┤ U2(0,π/2) ├───■────────────────────────────────────────■──────»
«     ├───────────┴┐┌─┴─┐                                    ┌─┴─┐    »
«q_1: ┤ U2(π/2,-π) ├┤ X ├──■──────────────────────────■──────┤ X ├────»
«     └────────────┘└───┘┌─┴─┐                      ┌─┴─┐┌───┴───┴───┐»
«q_2: ───────────────────┤ X ├──■────────────────■──┤ X ├┤ U2(0,π/2) ├»
«                        └───┘┌─┴─┐┌──────────┐┌─┴─┐├───┤└───────────┘»
«q_3: ────────────────────────┤ X ├┤ Rz(p[3]) ├┤ X ├┤ H ├─────────────»
«                             └───┘└──────────┘└───┘└───┘             »
«     ┌────────────┐                                                        »
«q_0: ┤ U2(π/2,-π) ├──■─────────────────────────────────────────────────────»
«     └───┬───┬────┘┌─┴─┐                                 ┌──────────┐      »
«q_1: ────┤ H ├─────┤ X ├──■──────────────────────────■───┤ Rx(-π/2) ├───■──»
«         └───┘     └───┘┌─┴─┐                      ┌─┴─┐┌┴──────────┴┐┌─┴─┐»
«q_2: ───────────────────┤ X ├──■────────────────■──┤ X ├┤ U2(π/2,-π) ├┤ X ├»
«                        └───┘┌─┴─┐┌──────────┐┌─┴─┐├───┤└────────────┘└───┘»
«q_3: ────────────────────────┤ X ├┤ Rz(p[4]) ├┤ X ├┤ H ├───────────────────»
«                             └───┘└──────────┘└───┘└───┘                   »
«                                                                             »
«q_0: ────────────────────────────────────────────────────────────────────────»
«                                       ┌─────────┐                           »
«q_1: ───────────────────────────■──────┤ Rx(π/2) ├──■────────────────────────»
«                              ┌─┴─┐    └─────────┘┌─┴─┐                      »
«q_2: ──■────────────────■─────┤ X ├───────────────┤ X ├──■────────────────■──»
«     ┌─┴─┐┌──────────┐┌─┴─┐┌──┴───┴───┐           └───┘┌─┴─┐┌──────────┐┌─┴─┐»
«q_3: ┤ X ├┤ Rz(p[5]) ├┤ X ├┤ Rx(-π/2) ├────────────────┤ X ├┤ Rz(p[6]) ├┤ X ├»
«     └───┘└──────────┘└───┘└──────────┘                └───┘└──────────┘└───┘»
«                     ┌───┐
«q_0: ─────────────■──┤ H ├
«                ┌─┴─┐└───┘
«q_1: ─────■─────┤ X ├─────
«        ┌─┴─┐   ├───┤     
«q_2: ───┤ X ├───┤ H ├─────
«     ┌──┴───┴──┐└───┘     
«q_3: ┤ Rx(π/2) ├──────────
«     └─────────┘

In [ ]:
def ranking_entanglers(qubit_op: PauliSumOp, gradient_Dict: dict,
                       sin_circuit, qmf_circuit: QuantumCircuit,
                       optimizer, backend) -> list:
    '''This function generate all the entanglers from gradient_dict and compute their Delta_E. Finally we
    select and output those with the largest magnitude.
    
    Args:
        qubit_op:
        qmf_state:
        gradient_Dict:
    Returns:
        entanglers: list object storing ranked entanglers with their delta_E in asending order.
        (delta_E being negative)
    '''
    Delta_E_Dict = {}
    for i in gradient_Dict.keys():
        
        # Not sure why at this moment
        if 'I' in i:
            Pauli_list = permute(i)
        else:
            Pauli_list = [i]
            
        for j in Pauli_list:
            energy = Delta_E(qubit_op, j, sin_circuit, qmf_circuit, optimizer, backend)
            Delta_E_Dict.update({j: energy})
    
    sorted_dict = dict(sorted(Delta_E_Dict.items(), key=lambda item: item[1], reverse=False))
    entanglers = list(sorted_dict.keys())
    
    return sorted_dict, entanglers

In [ ]:
hf_circuit = QuantumCircuit(4)
hf_circuit.x(0)
hf_circuit.x(1)

hf_circuit.draw()

In [ ]:
backend = Aer.get_backend("statevector_simulator")
distances = np.arange(0.6, 5.0, 0.2)
optimizer = COBYLA()

FCI = []
UCC = []
QCC = []

for dist in tqdm(distances):
    
    qubit_op, converter, problem = get_qubit_op(dist)
    num_particles = (1, 1)
    num_spin_orbitals = 6
    
    # Exact Diagonalization
    numpy_solver = NumPyMinimumEigensolver()
    calc = GroundStateEigensolver(converter, numpy_solver)
    res = calc.solve(problem)
    transformed_energy = res.extracted_transformer_energy
    nuclear_repulsion = res.nuclear_repulsion_energy
    FCI.append(np.real(res.total_energies))
    
    # Unitary Coupled Cluster
    ucc_ansatz = UCCSD(converter, num_particles, num_spin_orbitals, initial_state=hf_circuit)
    vqe_solver = VQE(ansatz=ucc_ansatz, optimizer=optimizer, quantum_instance=backend, callback = store_ucc_intermediate_result)
    calc = GroundStateEigensolver(converter, vqe_solver)
    res = calc.solve(problem)  
    UCC.append(np.real(res.total_energies))
    
    # Qubit Coupled Cluster; these entanglers are taken from the QCC paper
    entanglers = ['IXXY', 'ZYZX', 'IXYX', 'ZXXY', 'XYZX', 'ZXYX', 'YXZX']
    qcc_circuit = construct_qcc_circuit(entanglers, backend)
    ansatz = hf_circuit + qcc_circuit
    vqe_solver = VQE(ansatz=ansatz, optimizer=optimizer, quantum_instance=backend)
    calc = GroundStateEigensolver(converter, vqe_solver)
    res = calc.solve(problem)
    qcc_energy = np.real(res.total_energies) 
    QCC.append(qcc_energy)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6,4), dpi=200)
plt.plot(distances, FCI, label='FCI', linewidth=2, color='tab:blue')
plt.scatter(distances, QCC, label='QCC', marker='D', color='tab:orange', zorder=5)
plt.scatter(distances, UCC, label='UCC', s=150, edgecolor='green', facecolor='none', zorder=5)
plt.xlabel('Atomic distance (Angstrom)', fontsize=16)
plt.ylabel('Energy [Ha]', fontsize=16)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.tick_params(direction='in')
plt.title('Potential Energy Surface of LiH', fontsize=16)
plt.legend(fontsize=16, loc='best', edgecolor='black')

qcc_error = []
ucc_error = []
for i in range(len(distances)):
    qcc_error.append((QCC[i] - FCI[i])*1000)
    ucc_error.append((UCC[i] - FCI[i])*1000)

ax = plt.axes([0.3, 0.5, 0.3, 0.3], label='1')
ax.plot(distances, qcc_error, '--o', color='tab:orange', marker='D')
ax.plot(distances, ucc_error, '--o', color='green', mfc='none')
ax.plot(distances, [1.5]*len(distances), linestyle='dashed', color='tab:blue', linewidth=3)
ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis='y', labelsize=14)
ax.set_ylabel('mHa', fontsize=14)

plt.show()

In [ ]:
backend = Aer.get_backend("statevector_simulator")
distances = np.arange(0.6, 5.0, 0.2)
optimizer = COBYLA()

FCI = []
UCC = []
QCC = []

for dist in tqdm(distances):
    
    qubit_op, converter, problem = get_qubit_op(dist)
    num_particles = (1, 1)
    num_spin_orbitals = 6
    
    # Exact Diagonalization
    numpy_solver = NumPyMinimumEigensolver()
    calc = GroundStateEigensolver(converter, numpy_solver)
    res = calc.solve(problem)
    transformed_energy = res.extracted_transformer_energy
    nuclear_repulsion = res.nuclear_repulsion_energy
    FCI.append(np.real(res.total_energies))
    
    # Unitary Coupled Cluster
    ansatz = UCCSD(converter, num_particles, num_spin_orbitals, initial_state=hf_circuit)
    vqe_solver = VQE(ansatz=ansatz, optimizer=optimizer, quantum_instance=backend)
    calc = GroundStateEigensolver(converter, vqe_solver)
    res = calc.solve(problem)  
    UCC.append(np.real(res.total_energies))
    
    # Qubit Coupled Cluster; these entanglers are taken from the QCC paper
    entanglers = ['XXXY', 'IXXY', 'ZYZX', 'IXYX', 'ZXXY', 'XYZX', 'ZXYX', 'YXZX']
    qcc_circuit = construct_qcc_circuit(entanglers, backend)
    ansatz = hf_circuit + qcc_circuit
    vqe_solver = VQE(ansatz=ansatz, optimizer=optimizer, quantum_instance=backend)
    calc = GroundStateEigensolver(converter, vqe_solver)
    res = calc.solve(problem)
    qcc_energy = np.real(res.total_energies) 
    QCC.append(qcc_energy)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6,4), dpi=200)
plt.plot(distances, FCI, label='FCI', linewidth=2, color='tab:blue')
plt.scatter(distances, QCC, label='QCC', marker='D', color='tab:orange', zorder=5)
plt.scatter(distances, UCC, label='UCC', s=150, edgecolor='green', facecolor='none', zorder=5)
plt.xlabel('Atomic distance (Angstrom)', fontsize=16)
plt.ylabel('Energy [Ha]', fontsize=16)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.tick_params(direction='in')
plt.title('Potential Energy Surface of LiH', fontsize=16)
plt.legend(fontsize=16, loc='best', edgecolor='black')

qcc_error = []
ucc_error = []
for i in range(len(distances)):
    qcc_error.append((QCC[i] - FCI[i])*1000)
    ucc_error.append((UCC[i] - FCI[i])*1000)

ax = plt.axes([0.3, 0.5, 0.3, 0.3], label='1')
ax.plot(distances, qcc_error, '--o', color='tab:orange', marker='D')
ax.plot(distances, ucc_error, '--o', color='green', mfc='none')
ax.plot(distances, [1.5]*len(distances), linestyle='dashed', color='tab:blue', linewidth=3)
ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis='y', labelsize=14)
ax.set_ylabel('mHa', fontsize=14)

plt.show()

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table